In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
            
    def forward(self, x):
        out = self.left(x)
        out = out + self.shortcut(x)
        out = F.relu(out)
        
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)        
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)        
        self.fc = nn.Linear(512, num_classes, bias = False)
        
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = F.log_softmax(out)
        return out

In [3]:
def ResNet18():
    return ResNet(ResidualBlock)

In [4]:
#Use the ResNet18 on Cifar-10
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import argparse
import os
#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#set hyperparameter
EPOCH = 20
pre_epoch = 0
BATCH_SIZE = 500
LR = 1

#prepare dataset and preprocessing
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation((-7,7)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), 
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    transforms.RandomErasing(p=0.75,scale=(0.02, 0.1),value=1.0, inplace=False)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

#labels in CIFAR10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# #define ResNet18
net = ResNet18().to(device)

#define loss funtion & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=LR, betas=(0.9, 0.999))

Files already downloaded and verified
Files already downloaded and verified


In [102]:
# #train
# lr_decay_epochs=[20, 40]

# for epoch in range(pre_epoch, EPOCH+20):
#     print('\nEpoch: %d' % (epoch + 1))
#     net.train()
#     sum_loss = 0.0
#     correct = 0.0
#     total = 0.0
#     if epoch in lr_decay_epochs:
#         for param_group in optimizer.param_groups:
#             param_group['lr'] *=  0.5

#     for i, data in enumerate(trainloader, 0):
#         #prepare dataset
#         length = len(trainloader)
#         inputs, labels = data
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
        
#         #forward & backward
#         outputs = net(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         #print ac & loss in each batch
#         sum_loss += loss.item()
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += predicted.eq(labels.data).cpu().sum()
#         print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% ' 
#               % (epoch + 1, (i + 1 + (epoch) * length), sum_loss / (i + 1), 100. * correct / total))
        
#     #get the ac with testdataset in each epoch
#     print('Waiting Test...')
#     with torch.no_grad():
#         correct = 0
#         total = 0
#         for data in testloader:
#             net.eval()
#             images, labels = data
#             images, labels = images.to(device), labels.to(device)
#             outputs = net(images)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum()
#         print('Test\'s ac is: %.3f%%' % (100 * correct / total))

# print('Train has finished, total epoch is %d' % (EPOCH))

In [103]:
# FILE = 'resnet18_qt.pt'

# net = ResNet18().to(device)
# optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# checkpoint = torch.load(FILE)
# net.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# torch.save({'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, FILE)

In [23]:
#train
alpha=0.2
optimizer = optim.SGD(net.parameters(), lr=0.03, momentum=0.9)
lr_decay_epochs=[5, 10, 20, 30, 60, 70, 85]
for epoch in range(60, 100):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    epsilon=.05**((epoch-20)//20)
    if epoch in lr_decay_epochs:
        for param_group in optimizer.param_groups:
            param_group['lr'] *=  0.5

    for i, data in enumerate(trainloader, 0):
        #prepare dataset
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        #forward & backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        with torch.no_grad():
            for param_group in optimizer.param_groups:
                for idx, p in enumerate(param_group['params']):
                    constr = epsilon-(p.data**2-1)**2
                    Kx = -4 * (p.data**2 - 1) * p.data
                    direct_grad = torch.logical_or(torch.logical_or(constr >= 0, Kx == 0), torch.logical_and(constr < 0, (-Kx * p.grad.data) >= -alpha * constr))
                    p.grad.data[direct_grad] = p.grad.data[direct_grad]
                    p.grad.data[~direct_grad] = torch.clip(alpha * constr / Kx, -12, 12)[~direct_grad]
        optimizer.step()
        with torch.no_grad():
            for param_group in optimizer.param_groups:
                for idx, p in enumerate(param_group['params']):
                    torch.nan_to_num(p,0,+1,-1)
                    p.data.clamp_(-1.5,1.5)
        #print ac & loss in each batch
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% ' 
              % (epoch + 1, (i + 1 + (epoch) * length), sum_loss / (i + 1), 100. * correct / total))
        
    #get the ac with testdataset in each epoch
    print('Waiting Test...')
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:
            net.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('Test\'s ac is: %.3f%%' % (100 * correct / total))



Epoch: 61


C:\Users\lenovo\AppData\Local\Temp\ipykernel_8920\650345200.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.log_softmax(out)


[epoch:61, iter:6001] Loss: 5.585 | Acc: 10.800% 
[epoch:61, iter:6002] Loss: 4.959 | Acc: 12.900% 
[epoch:61, iter:6003] Loss: 4.928 | Acc: 12.933% 
[epoch:61, iter:6004] Loss: 4.684 | Acc: 12.950% 
[epoch:61, iter:6005] Loss: 4.620 | Acc: 12.960% 
[epoch:61, iter:6006] Loss: 4.593 | Acc: 12.867% 
[epoch:61, iter:6007] Loss: 4.562 | Acc: 12.857% 
[epoch:61, iter:6008] Loss: 4.479 | Acc: 12.900% 
[epoch:61, iter:6009] Loss: 4.487 | Acc: 12.956% 
[epoch:61, iter:6010] Loss: 4.401 | Acc: 12.880% 
[epoch:61, iter:6011] Loss: 4.393 | Acc: 13.000% 
[epoch:61, iter:6012] Loss: 4.354 | Acc: 13.067% 
[epoch:61, iter:6013] Loss: 4.319 | Acc: 13.154% 
[epoch:61, iter:6014] Loss: 4.280 | Acc: 13.386% 
[epoch:61, iter:6015] Loss: 4.233 | Acc: 13.267% 
[epoch:61, iter:6016] Loss: 4.234 | Acc: 13.438% 
[epoch:61, iter:6017] Loss: 4.259 | Acc: 13.388% 
[epoch:61, iter:6018] Loss: 4.228 | Acc: 13.322% 
[epoch:61, iter:6019] Loss: 4.216 | Acc: 13.368% 
[epoch:61, iter:6020] Loss: 4.193 | Acc: 13.410% 


KeyboardInterrupt: 

In [108]:
FILE = 'resnet18_qt.pt'

torch.save({'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, FILE)

In [ ]:
# net = ResNet18().to(device)
# optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
# FILE = 'resnet18_qt.pt'
# checkpoint = torch.load(FILE)
# net.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


with torch.no_grad():
    correct = 0
    total = 0
    net.eval()

    for name, param in net.named_parameters():
        print(param)
        param.data=torch.where(param.data>0, torch.Tensor([1]).to(device), torch.Tensor([-1]).to(device))
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Test\'s ac is: %.3f%%' % (100 * correct / total))


Parameter containing:
tensor([[[[ 0.9986,  1.0000, -1.0019],
          [ 0.9990,  1.0000,  0.9975],
          [ 0.9988,  0.9982,  0.9985]],

         [[-1.0024, -1.0028, -1.0013],
          [-1.0018, -1.0006, -1.0028],
          [-1.0009, -1.0019, -1.0010]],

         [[ 0.9984,  0.9960,  0.9964],
          [ 0.9984, -1.0036, -1.0005],
          [-1.0006,  0.9972,  0.9960]]],


        [[[ 1.0002,  1.0002,  1.0001],
          [-0.9968, -0.9999,  1.0015],
          [-0.9965, -0.9996, -0.9989]],

         [[ 1.0004,  1.0004,  1.0001],
          [ 1.0001,  0.9997,  1.0003],
          [-0.9991, -0.9999, -0.9992]],

         [[ 1.0022,  1.0007,  0.9997],
          [ 1.0004, -1.0003, -0.9979],
          [-0.9978, -0.9983, -1.0008]]],


        [[[-0.9999, -0.9999, -1.0000],
          [-1.0001, -1.0004, -1.0000],
          [-1.0007, -1.0003, -1.0001]],

         [[ 0.9994,  0.9982,  0.9996],
          [-1.0004, -1.0004,  0.9994],
          [-1.0007, -1.0012,  0.9991]],

         [[ 0.9982, -1

C:\Users\lenovo\AppData\Local\Temp\ipykernel_8920\650345200.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.log_softmax(out)


Test's ac is: 12.030%
